##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# El modelo Secuencial

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/keras/sequential_model">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/keras/sequential_model.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/keras/sequential_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/keras/sequential_model.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

## Preparación

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Cuándo utilizar un modelo Secuencial

Un modelo `Sequential` es apropiado para realizar **una simple pila de capas** donde cada capa tiene **exactamente un tensor de entrada y un tensor de salida**.

Esquemáticamente, el siguiente modelo `Sequential`:

In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

es equivalente a esta función:

In [ ]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

Un modelo Secuencial es **no apropiado** cuando:

- Su modelo tiene múltiples entradas o múltiples salidas
- Cualquiera de sus capas tiene múltiples entradas o múltiples salidas
- Es necesario compartir capas
- Desea una topología no lineal (por ejemplo, una conexión residual, un modelo multirramal)

## Crear un modelo Secuencial

Puede crear un modelo Secuencial pasando una lista de capas al constructor Secuencial:

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

Se puede acceder a sus capas mediante el atributo `layers`:

In [ ]:
model.layers

También puede crear un modelo Secuencial de forma incremental mediante el método `add()`:

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

Tenga en cuenta que también hay un método `pop()` para eliminar capas: un modelo Secuencial se comporta como una lista de capas.

In [ ]:
model.pop()
print(len(model.layers))  # 2

Tenga en cuenta también que el constructor Secuencial acepta un argumento `name`, al igual que cualquier capa o modelo en Keras. Esto es útil para anotar gráficos TensorBoard con nombres semánticamente significativos.

In [ ]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

## Especificación previa de la forma de entrada

Generalmente, todas las capas en Keras necesitan conocer la forma de sus entradas para poder crear sus pesos. Así que cuando cree una capa como esta, inicialmente, no tiene pesos:

In [ ]:
layer = layers.Dense(3)
layer.weights  # Empty

Para ello, creará sus ponderaciones la primera vez que se le llame con una entrada, ya que la forma de las ponderaciones depende de la forma de las entradas:

In [ ]:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

Naturalmente, esto también se aplica a los modelos secuenciales. Al crear una instancia de un modelo secuencial sin una forma de entrada, no se "construye": no tiene pesos (y al llamar a `model.weights` se produce un error que indica precisamente esto). Los pesos se crean cuando el modelo ve por primera vez algunos datos de entrada:

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

Una vez "construido" un modelo, puede llamar a su método `summary()` para mostrar su contenido:

In [ ]:
model.summary()

Sin embargo, puede ser muy útil cuando se construye un modelo secuencial de forma incremental para poder mostrar el resumen del modelo hasta el momento, incluyendo la forma de salida actual. En este caso, debe iniciar su modelo pasando un objeto `Input` a su modelo, para que conozca su forma de entrada desde el principio:

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

Tenga en cuenta que el objeto `Input` no se muestra como parte de `model.layers`, ya que no es una capa:

In [ ]:
model.layers

Una alternativa sencilla es pasar un argumento `input_shape` a la primera capa:

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

Los modelos construidos con una forma de entrada predefinida como esta siempre tienen pesos (incluso antes de ver cualquier dato) y siempre tienen una forma de salida definida.

En general, se recomienda especificar siempre por adelantado la forma de entrada de un modelo secuencial si se sabe cuál es.

## Un flujo de trabajo de depuración común: `add()` + `summary()`

Cuando se construye una nueva arquitectura secuencial, es útil apilar capas incrementalmente con `add()` e imprimir frecuentemente resúmenes del modelo. Por ejemplo, esto le permite controlar cómo una pila de capas `Conv2D` y `MaxPooling2D` está reduciendo el muestreo de los mapas de características de la imagen:

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

Es muy práctico, ¿verdad?


## Qué hacer una vez que se tiene un modelo

Una vez que su arquitectura modelo esté lista, querrá:

- Entrenar su modelo, evaluarlo y ejecutar la inferencia. Consulte nuestra [guía de entrenamiento y evaluación con los bucles incorporados](https://www.tensorflow.org/guide/keras/train_and_evaluate/)
- Guarde su modelo en disco y restáurelo. Consulte nuestra [guía de serialización y guardado](https://www.tensorflow.org/guide/keras/save_and_serialize/).
- Acelerar el entrenamiento de modelos aprovechando múltiples GPUs. Consulte nuestra [guía sobre multi-GPU y entrenamiento distribuido](https://keras.io/guides/distributed_training/).

## Extracción de características con un modelo secuencial

Una vez se ha construido un modelo Secuencial, se comporta como un [modelo API Funcional](https://www.tensorflow.org/guide/keras/functional/). Esto significa que cada capa tiene un atributo `input` y `output`. Estos atributos se pueden utilizar para hacer cosas interesantes, como crear rápidamente un modelo que extraiga las salidas de todas las capas intermedias en un modelo secuencial:

In [ ]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

Este es un ejemplo similar que solo extrae características de una capa:

In [ ]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

## Aprendizaje por transferencia con un modelo secuencial

El aprendizaje por transferencia consiste en congelar las capas inferiores de un modelo y entrenar únicamente las capas superiores. Si no está familiarizado con él, asegúrese de leer nuestra [guía del aprendizaje por transferencia](https://www.tensorflow.org/guide/keras/transfer_learning/).

A continuación se presentan dos modelos comunes de aprendizaje por transferencia que implican modelos secuenciales.

Primero, digamos que tiene un modelo Secuencial, y quiere congelar todas las capas excepto la última. En este caso, simplemente iteraría sobre `model.layers` y establecería `layer.trainable = False` en cada capa, excepto en la última. Así:

```python
model = keras.Sequential([
    keras.Input(shape=(784)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])

# Presumably you would want to first load pre-trained weights.
model.load_weights(...)

# Freeze all layers except the last one.
for layer in model.layers[:-1]:
  layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).
model.compile(...)
model.fit(...)
```

Otro modelo común es utilizar un modelo secuencial para apilar un modelo pre-entrenado y algunas capas de clasificación recién inicializadas. De esta manera:

```python
# Load a convolutional base with pre-trained weights
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

# Freeze the base model
base_model.trainable = False

# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([
    base_model,
    layers.Dense(1000),
])

# Compile & train
model.compile(...)
model.fit(...)
```

Si se dedica al aprendizaje por transferencia, es probable que utilice con frecuencia estos dos patrones.

Esto es todo lo que necesita saber sobre los modelos secuenciales.

Para obtener más información sobre la creación de modelos en Keras, consulte:

- [Guía de la API Funcional](https://www.tensorflow.org/guide/keras/functional/)
- [Guía para crear nuevas capas y modelos mediante subclases](https://www.tensorflow.org/guide/keras/custom_layers_and_models/)